In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-01-30 19:49:21--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-01-30 19:49:21 (6.54 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("FinalProject").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
import pandas as pd
# Read in Zillow data from S3 Buckets
from pyspark import SparkFiles
url = "https://housingdata21.s3.us-east-2.amazonaws.com/Clean_ZillowData.csv"
spark.sparkContext.addFile(url)
zillow_df = spark.read.csv(SparkFiles.get("Clean_ZillowData.csv"), sep=",", header=True, inferSchema=True)
zillow_df.show()

+------------------+-------------------+
|       Region Name|Average ZHVI (2018)|
+------------------+-------------------+
|       Cook County|          251566.75|
|     DuPage County|           324148.0|
|       Lake County|        252448.5833|
|       Will County|        222828.4167|
|       Kane County|        239270.9167|
|    McHenry County|        223408.9167|
|  Winnebago County|           108585.5|
|Saint Clair County|        110394.9167|
|    Madison County|        122395.0833|
|  Champaign County|        143673.0833|
|   Sangamon County|        133052.5833|
|     Peoria County|        102460.6667|
|     McLean County|        157759.6667|
|Rock Island County|           107559.0|
|   Tazewell County|        122630.4167|
|    Kendall County|          238588.75|
|   La Salle County|        121535.0833|
|   Kankakee County|        138533.9167|
|      Macon County|        90090.83333|
|     DeKalb County|           167234.5|
+------------------+-------------------+
only showing top

In [5]:
from functools import reduce

oldColumns = zillow_df.schema.names
newColumns = ["region_name", "zhvi"]

zillow_df = reduce(lambda zillow_df, idx: zillow_df.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)), zillow_df)
zillow_df.printSchema()
zillow_df.show()

root
 |-- region_name: string (nullable = true)
 |-- zhvi: double (nullable = true)

+------------------+-----------+
|       region_name|       zhvi|
+------------------+-----------+
|       Cook County|  251566.75|
|     DuPage County|   324148.0|
|       Lake County|252448.5833|
|       Will County|222828.4167|
|       Kane County|239270.9167|
|    McHenry County|223408.9167|
|  Winnebago County|   108585.5|
|Saint Clair County|110394.9167|
|    Madison County|122395.0833|
|  Champaign County|143673.0833|
|   Sangamon County|133052.5833|
|     Peoria County|102460.6667|
|     McLean County|157759.6667|
|Rock Island County|   107559.0|
|   Tazewell County|122630.4167|
|    Kendall County|  238588.75|
|   La Salle County|121535.0833|
|   Kankakee County|138533.9167|
|      Macon County|90090.83333|
|     DeKalb County|   167234.5|
+------------------+-----------+
only showing top 20 rows



In [6]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://finalproject.c969jxlkzz2v.us-east-2.rds.amazonaws.com:5432/housingdata"
config = {"user":"postgres",
          "password": "ilovegroup3",
          "driver":"org.postgresql.Driver"}

In [7]:
# Write DataFrame to zillow_data table in RDS
zillow_df.write.jdbc(url=jdbc_url, table='zillow_data', mode=mode, properties=config)

In [8]:
# Read in ACS data from S3 Buckets
url ="https://housingdata21.s3.us-east-2.amazonaws.com/Clean_ACSData.csv"
spark.sparkContext.addFile(url)
acs_df = spark.read.csv(SparkFiles.get("Clean_ACSData.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
acs_df.show(50)

+--------------------+------------+----------------+-----------+-------------+-------------+-----------+---------------+--------------+-----------+--------------+------------+--------------+--------------+-------------+-------------+------------------+---------------+----------------+---------------+----------------+-----------+-----------------+----------------+
|                 _c0|Adams County|Champaign County|Cook County|DeKalb County|DuPage County|Kane County|Kankakee County|Kendall County|Lake County|LaSalle County|Macon County|Madison County|McHenry County|McLean County|Peoria County|Rock Island County|Sangamon County|St. Clair County|Tazewell County|Vermilion County|Will County|Williamson County|Winnebago County|
+--------------------+------------+----------------+-----------+-------------+-------------+-----------+---------------+--------------+-----------+--------------+------------+--------------+--------------+-------------+-------------+------------------+---------------+

In [9]:
# Transpose ACS DataFrame
acs_df = acs_df.toPandas().set_index("_c0").transpose()
acs_df

_c0,Total population,Male,Female,Median age (years),White,Black or African American,American Indian and Alaska Native,Asian,Native Hawaiian and Other Pacific Islander,Some other race,Hispanic or Latino (of any race),Not Hispanic or Latino,Median income (dollars),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0
Adams County,"65,691","32,661","33,030",41.5,"61,932","3,375",N,813,N,N,"1,091","64,600",56699,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0,0,0,0,0,0,0,0,0,0,0,0,0
Champaign County,"209,983","105,260","104,723",30.3,"155,268","31,162",1174,"25,698",N,"2,713","12,798","197,185",52725,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0,0,0,0,0,0,0,0,0,0,0,0,0
Cook County,"5,180,493","2,516,466","2,664,027",37,"3,077,682","1,258,171","41,587","445,378",6874,"501,045","1,323,017","3,857,476","63,353",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0,0,0,0,0,0,0,0,0,0,0,0,0
DeKalb County,"104,143","50,903","53,240",30.9,"92,165","10,105",258,"3,644",N,"1,390","11,921","92,222","63,085",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0,0,0,0,0,0,0,0,0,0,0,0,0
DuPage County,"928,589","456,716","471,873",39.5,"728,595","53,896","7,481","125,861","2,889","39,641","134,231","794,358","93,750",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0,0,0,0,0,0,0,0,0,0,0,0,0
Kane County,"534,216","266,324","267,892",37.7,"416,718","34,111","4,546","27,668",639,"66,943","172,268","361,948","79,818",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0,0,0,0,0,0,0,0,0,0,0,0,0
Kankakee County,"110,024","54,768","55,256",37.5,"90,134","17,867",872,"1,489",N,"3,015","11,691","98,333","51,404",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0,0,0,0,0,0,0,0,0,0,0,0,0
Kendall County,"127,915","62,401","65,514",35.2,"111,982","11,529",N,"5,816",N,"1,898","24,899","103,016","89,862",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0,0,0,0,0,0,0,0,0,0,0,0,0
Lake County,"700,832","350,487","350,345",38.5,"536,569","56,192",7069,"63,592",1138,"59,061","155,394","545,438","87,133",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0,0,0,0,0,0,0,0,0,0,0,0,0
LaSalle County,"109,430","55,242","54,188",41.9,"102,609","4,677",774,"1,294",N,"2,567","10,972","98,458","51,838",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
#Drop all NaN columns
acs_df = acs_df.dropna(axis=1, how='all')
acs_df

_c0,Total population,Male,Female,Median age (years),White,Black or African American,American Indian and Alaska Native,Asian,Native Hawaiian and Other Pacific Islander,Some other race,Hispanic or Latino (of any race),Not Hispanic or Latino,Median income (dollars),0,0,0,0,0,0,0,0,0,0,0,0,0
Adams County,"65,691","32,661","33,030",41.5,"61,932","3,375",N,813,N,N,"1,091","64,600",56699,0,0,0,0,0,0,0,0,0,0,0,0,0
Champaign County,"209,983","105,260","104,723",30.3,"155,268","31,162",1174,"25,698",N,"2,713","12,798","197,185",52725,0,0,0,0,0,0,0,0,0,0,0,0,0
Cook County,"5,180,493","2,516,466","2,664,027",37,"3,077,682","1,258,171","41,587","445,378",6874,"501,045","1,323,017","3,857,476","63,353",0,0,0,0,0,0,0,0,0,0,0,0,0
DeKalb County,"104,143","50,903","53,240",30.9,"92,165","10,105",258,"3,644",N,"1,390","11,921","92,222","63,085",0,0,0,0,0,0,0,0,0,0,0,0,0
DuPage County,"928,589","456,716","471,873",39.5,"728,595","53,896","7,481","125,861","2,889","39,641","134,231","794,358","93,750",0,0,0,0,0,0,0,0,0,0,0,0,0
Kane County,"534,216","266,324","267,892",37.7,"416,718","34,111","4,546","27,668",639,"66,943","172,268","361,948","79,818",0,0,0,0,0,0,0,0,0,0,0,0,0
Kankakee County,"110,024","54,768","55,256",37.5,"90,134","17,867",872,"1,489",N,"3,015","11,691","98,333","51,404",0,0,0,0,0,0,0,0,0,0,0,0,0
Kendall County,"127,915","62,401","65,514",35.2,"111,982","11,529",N,"5,816",N,"1,898","24,899","103,016","89,862",0,0,0,0,0,0,0,0,0,0,0,0,0
Lake County,"700,832","350,487","350,345",38.5,"536,569","56,192",7069,"63,592",1138,"59,061","155,394","545,438","87,133",0,0,0,0,0,0,0,0,0,0,0,0,0
LaSalle County,"109,430","55,242","54,188",41.9,"102,609","4,677",774,"1,294",N,"2,567","10,972","98,458","51,838",0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
#Drop all columns with "0"
acs_df = acs_df.drop(['0'], axis=1)
acs_df

_c0,Total population,Male,Female,Median age (years),White,Black or African American,American Indian and Alaska Native,Asian,Native Hawaiian and Other Pacific Islander,Some other race,Hispanic or Latino (of any race),Not Hispanic or Latino,Median income (dollars)
Adams County,"65,691","32,661","33,030",41.5,"61,932","3,375",N,813,N,N,"1,091","64,600",56699
Champaign County,"209,983","105,260","104,723",30.3,"155,268","31,162",1174,"25,698",N,"2,713","12,798","197,185",52725
Cook County,"5,180,493","2,516,466","2,664,027",37,"3,077,682","1,258,171","41,587","445,378",6874,"501,045","1,323,017","3,857,476","63,353"
DeKalb County,"104,143","50,903","53,240",30.9,"92,165","10,105",258,"3,644",N,"1,390","11,921","92,222","63,085"
DuPage County,"928,589","456,716","471,873",39.5,"728,595","53,896","7,481","125,861","2,889","39,641","134,231","794,358","93,750"
Kane County,"534,216","266,324","267,892",37.7,"416,718","34,111","4,546","27,668",639,"66,943","172,268","361,948","79,818"
Kankakee County,"110,024","54,768","55,256",37.5,"90,134","17,867",872,"1,489",N,"3,015","11,691","98,333","51,404"
Kendall County,"127,915","62,401","65,514",35.2,"111,982","11,529",N,"5,816",N,"1,898","24,899","103,016","89,862"
Lake County,"700,832","350,487","350,345",38.5,"536,569","56,192",7069,"63,592",1138,"59,061","155,394","545,438","87,133"
LaSalle County,"109,430","55,242","54,188",41.9,"102,609","4,677",774,"1,294",N,"2,567","10,972","98,458","51,838"


In [12]:
acs_df.reset_index(level=0, inplace=True) 
acs_df

_c0,index,Total population,Male,Female,Median age (years),White,Black or African American,American Indian and Alaska Native,Asian,Native Hawaiian and Other Pacific Islander,Some other race,Hispanic or Latino (of any race),Not Hispanic or Latino,Median income (dollars)
0,Adams County,"65,691","32,661","33,030",41.5,"61,932","3,375",N,813,N,N,"1,091","64,600",56699
1,Champaign County,"209,983","105,260","104,723",30.3,"155,268","31,162",1174,"25,698",N,"2,713","12,798","197,185",52725
2,Cook County,"5,180,493","2,516,466","2,664,027",37,"3,077,682","1,258,171","41,587","445,378",6874,"501,045","1,323,017","3,857,476","63,353"
3,DeKalb County,"104,143","50,903","53,240",30.9,"92,165","10,105",258,"3,644",N,"1,390","11,921","92,222","63,085"
4,DuPage County,"928,589","456,716","471,873",39.5,"728,595","53,896","7,481","125,861","2,889","39,641","134,231","794,358","93,750"
5,Kane County,"534,216","266,324","267,892",37.7,"416,718","34,111","4,546","27,668",639,"66,943","172,268","361,948","79,818"
6,Kankakee County,"110,024","54,768","55,256",37.5,"90,134","17,867",872,"1,489",N,"3,015","11,691","98,333","51,404"
7,Kendall County,"127,915","62,401","65,514",35.2,"111,982","11,529",N,"5,816",N,"1,898","24,899","103,016","89,862"
8,Lake County,"700,832","350,487","350,345",38.5,"536,569","56,192",7069,"63,592",1138,"59,061","155,394","545,438","87,133"
9,LaSalle County,"109,430","55,242","54,188",41.9,"102,609","4,677",774,"1,294",N,"2,567","10,972","98,458","51,838"


In [13]:
column_indices = [0,1,2,3,4,5,6,7,8,9,10,11,12,13]
new_names = ["county","total_pop","male","female","median_age","race_white","race_black","race_nativeam","race_asian","race_nativehaw","race_other","eth_hispanic","eth_nonhispanic","median_income"]
old_names = acs_df.columns[column_indices]
acs_df.rename(columns=dict(zip(old_names, new_names)), inplace=True)
acs_df

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


_c0,county,total_pop,male,female,median_age,race_white,race_black,race_nativeam,race_asian,race_nativehaw,race_other,eth_hispanic,eth_nonhispanic,median_income
0,Adams County,"65,691","32,661","33,030",41.5,"61,932","3,375",N,813,N,N,"1,091","64,600",56699
1,Champaign County,"209,983","105,260","104,723",30.3,"155,268","31,162",1174,"25,698",N,"2,713","12,798","197,185",52725
2,Cook County,"5,180,493","2,516,466","2,664,027",37,"3,077,682","1,258,171","41,587","445,378",6874,"501,045","1,323,017","3,857,476","63,353"
3,DeKalb County,"104,143","50,903","53,240",30.9,"92,165","10,105",258,"3,644",N,"1,390","11,921","92,222","63,085"
4,DuPage County,"928,589","456,716","471,873",39.5,"728,595","53,896","7,481","125,861","2,889","39,641","134,231","794,358","93,750"
5,Kane County,"534,216","266,324","267,892",37.7,"416,718","34,111","4,546","27,668",639,"66,943","172,268","361,948","79,818"
6,Kankakee County,"110,024","54,768","55,256",37.5,"90,134","17,867",872,"1,489",N,"3,015","11,691","98,333","51,404"
7,Kendall County,"127,915","62,401","65,514",35.2,"111,982","11,529",N,"5,816",N,"1,898","24,899","103,016","89,862"
8,Lake County,"700,832","350,487","350,345",38.5,"536,569","56,192",7069,"63,592",1138,"59,061","155,394","545,438","87,133"
9,LaSalle County,"109,430","55,242","54,188",41.9,"102,609","4,677",774,"1,294",N,"2,567","10,972","98,458","51,838"


In [14]:
import pandas as pd

In [22]:
#Remove commas from acs_df
acs_df = acs_df.replace(',','', regex=True)
#Change N values to 0 in the acs_df
acs_df = acs_df.replace('N','0', regex=True)

In [23]:
#Change all columns to int values (except county which remains str)
acs_df[["total_pop","male","female","median_age","race_white","race_black","race_nativeam","race_asian","race_nativehaw","race_other","eth_hispanic","eth_nonhispanic","median_income"]] = acs_df[["total_pop","male","female","median_age","race_white","race_black","race_nativeam","race_asian","race_nativehaw","race_other","eth_hispanic","eth_nonhispanic","median_income"]].apply(pd.to_numeric)


In [24]:
acs_df = spark.createDataFrame(acs_df)
acs_df.show()

+------------------+---------+-------+-------+----------+----------+----------+-------------+----------+--------------+----------+------------+---------------+-------------+
|            county|total_pop|   male| female|median_age|race_white|race_black|race_nativeam|race_asian|race_nativehaw|race_other|eth_hispanic|eth_nonhispanic|median_income|
+------------------+---------+-------+-------+----------+----------+----------+-------------+----------+--------------+----------+------------+---------------+-------------+
|      Adams County|    65691|  32661|  33030|      41.5|     61932|      3375|            0|       813|             0|         0|        1091|          64600|        56699|
|  Champaign County|   209983| 105260| 104723|      30.3|    155268|     31162|         1174|     25698|             0|      2713|       12798|         197185|        52725|
|       Cook County|  5180493|2516466|2664027|      37.0|   3077682|   1258171|        41587|    445378|          6874|    501045|

In [25]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://finalproject.c969jxlkzz2v.us-east-2.rds.amazonaws.com:5432/housingdata"
config = {"user":"postgres",
          "password": "ilovegroup3",
          "driver":"org.postgresql.Driver"}

In [26]:
# Write DataFrame to acs_data table in RDS
acs_df.write.jdbc(url=jdbc_url, table='acs_data', mode=mode, properties=config)